In [4]:
import pandas as pd
import numpy as np

df = pd.read_csv(r"C:\Users\archit\Desktop\customer churn project\WA_Fn-UseC_-Telco-Customer-Churn.csv")

In [6]:
print(df.head())

   customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0  7590-VHVEG  Female              0     Yes         No       1           No   
1  5575-GNVDE    Male              0      No         No      34          Yes   
2  3668-QPYBK    Male              0      No         No       2          Yes   
3  7795-CFOCW    Male              0      No         No      45           No   
4  9237-HQITU  Female              0      No         No       2          Yes   

      MultipleLines InternetService OnlineSecurity  ... DeviceProtection  \
0  No phone service             DSL             No  ...               No   
1                No             DSL            Yes  ...              Yes   
2                No             DSL            Yes  ...               No   
3  No phone service             DSL            Yes  ...              Yes   
4                No     Fiber optic             No  ...               No   

  TechSupport StreamingTV StreamingMovies        Contract Pape

In [9]:
print(df.isnull().sum())


customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64


In [10]:
print(df["Churn"].value_counts())

Churn
No     5174
Yes    1869
Name: count, dtype: int64


In [11]:
print(df.isnull().values.any())


False


In [12]:
print(df['TotalCharges'].dtype)

object


In [13]:
print((df['TotalCharges'] == " ").sum())

11


In [16]:
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"],errors = 'coerce')

In [17]:
print((df['TotalCharges'] == " ").sum())

0


In [18]:
X = df.drop('Churn', axis=1)
y = df['Churn']


In [20]:
print(X)

      customerID  gender  SeniorCitizen Partner Dependents  tenure  \
0     7590-VHVEG  Female              0     Yes         No       1   
1     5575-GNVDE    Male              0      No         No      34   
2     3668-QPYBK    Male              0      No         No       2   
3     7795-CFOCW    Male              0      No         No      45   
4     9237-HQITU  Female              0      No         No       2   
...          ...     ...            ...     ...        ...     ...   
7038  6840-RESVB    Male              0     Yes        Yes      24   
7039  2234-XADUH  Female              0     Yes        Yes      72   
7040  4801-JZAZL  Female              0     Yes        Yes      11   
7041  8361-LTMKD    Male              1     Yes         No       4   
7042  3186-AJIEK    Male              0      No         No      66   

     PhoneService     MultipleLines InternetService OnlineSecurity  \
0              No  No phone service             DSL             No   
1             Yes  

In [22]:
numerical_cols = X.select_dtypes(include =['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include = ['object']).columns

In [23]:
print(numerical_cols)

Index(['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges'], dtype='object')


In [24]:
print(categorical_cols)

Index(['customerID', 'gender', 'Partner', 'Dependents', 'PhoneService',
       'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup',
       'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
       'Contract', 'PaperlessBilling', 'PaymentMethod'],
      dtype='object')


In [25]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

In [26]:
num_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])


In [27]:
cat_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])


In [28]:
num_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])


In [29]:
preprocessor = ColumnTransformer(transformers=[
    ('num', num_pipeline, numerical_cols),
    ('cat', cat_pipeline, categorical_cols)
])

In [30]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


In [31]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000))
])


In [33]:
model.fit(X_train, y_train)

,steps,"[('preprocessor', ...), ('classifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [34]:
y_pred = model.predict(X_test)

In [36]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[924, 111],
       [166, 208]])

In [37]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

          No       0.85      0.89      0.87      1035
         Yes       0.65      0.56      0.60       374

    accuracy                           0.80      1409
   macro avg       0.75      0.72      0.73      1409
weighted avg       0.80      0.80      0.80      1409



In [38]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred, pos_label='Yes')


0.6002886002886003

In [41]:
from sklearn.metrics import make_scorer, f1_score
from sklearn.model_selection import cross_val_score

f1_scorer = make_scorer(f1_score, pos_label='Yes')

cv_scores = cross_val_score(
    model,
    X,
    y,
    cv=5,
    scoring=f1_scorer
)

print("Cross-Validation F1 Scores:", cv_scores)
print("Average CV F1 Score:", cv_scores.mean())


Cross-Validation F1 Scores: [0.59796807 0.61714286 0.57142857 0.61760462 0.59347181]
Average CV F1 Score: 0.5995231851862499


In [45]:
print(cv_scores)
print(cv_scores.mean())


[0.59796807 0.61714286 0.57142857 0.61760462 0.59347181]
0.5995231851862499


In [46]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, f1_score
from sklearn.model_selection import cross_val_score


In [47]:
rf_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(
        n_estimators=200,
        random_state=42,
        class_weight='balanced'
    ))
])


In [48]:
rf_model.fit(X_train, y_train)


,steps,"[('preprocessor', ...), ('classifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [49]:
rf_pred = rf_model.predict(X_test)

print("Random Forest Classification Report:")
print(classification_report(rf_pred, y_test))


Random Forest Classification Report:
              precision    recall  f1-score   support

          No       0.90      0.83      0.87      1119
         Yes       0.51      0.65      0.57       290

    accuracy                           0.80      1409
   macro avg       0.70      0.74      0.72      1409
weighted avg       0.82      0.80      0.81      1409



In [50]:
rf_f1 = f1_score(y_test, rf_pred, pos_label='Yes')
print("Random Forest F1 Score:", rf_f1)


Random Forest F1 Score: 0.5692771084337349


In [52]:
f1_scorer = make_scorer(f1_score, pos_label='Yes')

rf_cv_scores = cross_val_score(
    rf_model,
    X,
    y,
    cv=5,
    scoring=f1_scorer
)

print("Random Forest CV F1 Scores:", rf_cv_scores)
print("Average RF CV F1 Score:", rf_cv_scores.mean())


Random Forest CV F1 Scores: [0.56441718 0.56363636 0.51818182 0.58912387 0.56316591]
Average RF CV F1 Score: 0.5597050264866875
